<img src="http://oproject.org/img/ROOT.png" height="30%" width="30%">
<img src="http://oproject.org/img/tmvalogo.png" height="30%" width="30%">

<hr style="border-top-width: 4px; border-top-color: #34609b;">

# DNN Example

## Declare Factory

In [ ]:
TFile* inputFile = TFile::Open("data/inputdata.root");
TFile* outputFile = TFile::Open("TMVAOutput.root", "RECREATE");

TMVA::Factory factory("TMVAClassification", outputFile,
                      "!V:ROC:!Correlations:!Silent:Color:!DrawProgressBar:AnalysisType=Classification" ); 

## Declare DataLoader

In [ ]:
TMVA::DataLoader loader("dataset_dnn");

loader.AddVariable("var1");
loader.AddVariable("var2");
loader.AddVariable("var3");
loader.AddVariable("var4");

## Setup Dataset(s)

In [ ]:
TTree *tsignal = (TTree*) inputFile->Get("Sig");
TTree *tbackground = (TTree*) inputFile->Get("Bkg");

loader.AddSignalTree(tsignal);
loader.AddBackgroundTree(tbackground);
loader.PrepareTrainingAndTestTree("",
        "nTrain_Signal=1000:nTrain_Background=1000:SplitMode=Random:NormMode=NumEvents:!V"); 

# Configure network layout 

In [ ]:
// General layout
TString inputLayoutString = "InputLayout=1|1|4"; 
TString batchLayoutString= "BatchLayout=1|128|4";
TString layoutString ("Layout=DENSE|32|RELU,DENSE|32|RELU,DENSE|32|RELU,DENSE|32|RELU,DENSE|1|LINEAR");

// Training strategies 
// one can catenate several training strategies 
TString training1("LearningRate=1e-3,Momentum=0.9,Repetitions=1,"
                        "ConvergenceSteps=20,BatchSize=128,TestRepetitions=1,"
                        "MaxEpochs=20,WeightDecay=1e-4,Regularization=L2,"
                        "Optimizer=ADAM,DropConfig=0.0+0.0+0.0+0.");
  
TString trainingStrategyString ("TrainingStrategy=");
trainingStrategyString += training1; // + "|" + training2 + "|" + training3;

// General Options.                                                                                                                                                                
TString dnnOptions ("!H:V:ErrorStrategy=CROSSENTROPY:VarTransform=None:"
                          "WeightInitialization=XAVIERUNIFORM");
dnnOptions.Append (":"); dnnOptions.Append (inputLayoutString);
dnnOptions.Append (":"); dnnOptions.Append (batchLayoutString);
dnnOptions.Append (":"); dnnOptions.Append (layoutString);
dnnOptions.Append (":"); dnnOptions.Append (trainingStrategyString);

dnnOptions += ":Architecture=Standard";

# Booking Methods

In [ ]:


// Multi-Layer Perceptron (Neural Network)
factory.BookMethod(&loader, TMVA::Types::kMLP, "MLP",
        "!H:!V:NeuronType=tanh:VarTransform=N:NCycles=10:HiddenLayers=N+5:TestRate=5:!UseRegulator");

// Adaptive Boost Trees
factory.BookMethod(&loader, TMVA::Types::kBDT, "BDT",
                           "!H:!V:NTrees=850:MinNodeSize=2.5%:MaxDepth=3:BoostType=AdaBoost:AdaBoostBeta=0.5:UseBaggedBoost:BaggedSampleFraction=0.5:SeparationType=GiniIndex:nCuts=20" );


//Deep Neural Network
factory.BookMethod(&loader, TMVA::Types::kDL, "DL_DENSE", dnnOptions);



## Train Methods

In [ ]:
factory.TrainAllMethods();

## Test and Evaluate Methods

In [ ]:
factory.TestAllMethods();
factory.EvaluateAllMethods();    

## Plot ROC Curve
We enable JavaScript visualisation for the plots

In [ ]:
%jsroot on

In [ ]:
auto c2 = factory.GetROCCurve(&loader);
c2->Draw();